In [1]:
import cupy as cp
import numpy as np
import pandas as pd
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import scipy.stats as st
import datawig as dw
import mxnet as mx
import optuna
import re

from lightgbm import LGBMClassifier

from sklearn import metrics
from sklearn_pandas import DataFrameMapper
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss
from sklearn.metrics import mean_squared_log_error
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.experimental import enable_iterative_imputer
from sklearn.metrics import roc_auc_score
from sklearn.tree import plot_tree
from sklearn.datasets import make_moons
from sklearn.feature_selection import mutual_info_classif
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import mutual_info_classif
from sklearn.experimental import enable_iterative_imputer



from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import SimpleImputer

from keras.layers import Dense, Dropout
from keras.models import Sequential

from six import StringIO

from IPython.display import Image
import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm
import math
from scipy import stats
import numpy as np
import pandas as pd
import tensorflow as tf

from datetime import datetime
from tensorflow import feature_column
from tensorflow.keras import layers

from tqdm import tqdm

# sklearn preprocessing for dealing with categorical variables
from sklearn.preprocessing import LabelEncoder

# File system manangement
import os

# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')

import xgboost as xgb
from sklearn.metrics import log_loss
from xgboost.sklearn import XGBClassifier

from Modules import Modules
from Baseline import Baseline
from FeatureEngineering2 import FeatureEngineering2
from FeatureEngineering3 import FeatureEngineering3


%matplotlib inline

print("OK\n")

OK



In [2]:
baseline = Baseline('TARGET')
modules = Modules('TARGET')
FeatureEngineering2 = FeatureEngineering2('TARGET')
FeatureEngineering3 = FeatureEngineering3('TARGET')

In [3]:
"""
params_base = {
            'boosting_type': 'gbdt'
            ,'objective': 'binary'
            ,'metric': 'auc'
            ,'verbosity': -1
            ,'learning_rate': 0.05
            ,'n_estimators': 100000
            ,'random_state':123
            ,'bagging_freq': 1   
}
"""
params_base = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'learning_rate': 0.02
    ,'max_bin':400
    ,'max_depth': -1
    ,'num_leaves': 30
    ,'min_child_samples': 70
    ,'subsample': 1.0
    ,'subsample_freq': 1
    ,'colsample_bytree': 0.05
    ,'min_split_gain': 0.5
    ,'reg_alpha': 0.0
    ,'reg_lambda': 100
    ,'n_estimators': 10000
}

def objective(trial):
    # 探索するパラメータ
    param_tuning = {
            "num_leaves": trial.suggest_int("num_leaves", 8, 256)
            ,"min_child_samples": trial.suggest_int("min_child_samples", 5, 200)
            ,"min_sum_hessian_in_leaf": trial.suggest_float("min_sum_hessian_in_leaf", 1e-5, 1e-2, log=True)
            ,"feature_fraction": trial.suggest_float("feature_fraction", 0.5, 1.0)
            ,"bagging_fraction": trial.suggest_float("bagging_fraction", 0.5, 1.0)
            ,"lambda_l1": trial.suggest_float("lambda_l1", 1e-2, 1e+2, log=True)
            ,"lambda_l2": trial.suggest_float("lambda_l2", 1e-2, 1e+2, log=True)
    }
    param_tuning.update(params_base)

    # モデル学習・評価:
    list_metrics = []
    cv = list(StratifiedKFold(n_splits=5, shuffle=True, random_state=123).split(x_train, y_train))
    list_fold = [0]
    for nfold in list_fold:
        idx_tr, idx_va = cv[nfold][0], cv[nfold][1]
        x_tr, y_tr = x_train.loc[idx_tr, :], y_train[idx_tr]
        x_va, y_va = x_train.loc[idx_va, :], y_train[idx_va]
        model = lgb.LGBMClassifier(**params)
        model.fit(x_tr
                    ,y_tr
                    ,eval_set=[(x_tr, y_tr), (x_va, y_va)]
                    ,early_stopping_rounds=100
                    ,verbose=0
        )
        y_va_pred = model.predict_proba(x_va)[:,1]
        metric_va = roc_auc_score(y_va, y_va_pred)
        list_metrics.append(metric_va)

    # 評価指数の算出
    metrics = np.mean(list_metrics)

    return metrics

In [4]:
"""
params = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'learning_rate': 0.02
    ,'max_bin':400
    ,'max_depth': -1
    ,'num_leaves': 30
    ,'min_child_samples': 70
    ,'subsample': 1.0
    ,'subsample_freq': 1
    ,'colsample_bytree': 0.05
    ,'min_split_gain': 0.5
    ,'reg_alpha': 0.0
    ,'reg_lambda': 100
    ,'n_estimators': 2000
}
"""
"""
# Case2
params = {
    #'boosting_type': 'goss'
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'nthread':4
    ,'n_estimators':2000
    ,'learning_rate':0.005134
    ,'num_leaves':54
    ,'colsample_bytree':0.508716
    ,'subsample':1
    ,'max_depth':10
    ,'reg_alpha':0.436193
    ,'reg_lambda':0.479169
    ,'min_split_gain':0.024766
    ,'min_child_weight':40
    ,'silent':-1
    ,'verbose':-1
    ,'is_unbalance':False
}
"""
# Case3 2022/07/29
params_base = {
    'boosting_type': 'gbdt'
    ,'objective': 'binary'
    ,'metric': 'auc'
    ,'learning_rate': 0.02
    ,'max_bin':400
    ,'max_depth': -1
    ,'num_leaves': 30
    ,'min_child_samples': 70
    ,'subsample': 1.0
    ,'subsample_freq': 1
    ,'colsample_bytree': 0.05
    ,'min_split_gain': 0.5
    ,'reg_alpha': 0.0
    ,'reg_lambda': 100
    ,'n_estimators': 10000
}

In [5]:
apps, prev, bureau, bureau_bal, pos_bal, install, card_bal = FeatureEngineering3.get_dataset()
apps.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1.0,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0.0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0.0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0.0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0.0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
df = FeatureEngineering3.get_apps_all_with_all_agg(apps, prev, bureau, bureau_bal, pos_bal, install, card_bal)
df.head()

prev_agg shape: (338857, 41) bureau_agg shape: (305811, 103)
pos_bal_agg shape: (337252, 45) install_agg shape: (339587, 59) card_bal_agg shape: (103558, 70)
apps_all before merge shape: (356255, 152)
apps_all after merge with all shape: (356255, 466)


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CARD_M3BALANCE_LIMIT_RATIO_MIN,CARD_M3BALANCE_LIMIT_RATIO_MAX,CARD_M3DRAWING_LIMIT_RATIO_MIN,CARD_M3DRAWING_LIMIT_RATIO_MAX,CARD_M3CARD_IS_DPD_MEAN,CARD_M3CARD_IS_DPD_SUM,CARD_M3CARD_IS_DPD_UNDER_120_MEAN,CARD_M3CARD_IS_DPD_UNDER_120_SUM,CARD_M3CARD_IS_DPD_OVER_120_MEAN,CARD_M3CARD_IS_DPD_OVER_120_SUM
0,100002,1.0,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100003,0.0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100004,0.0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100006,0.0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,100007,0.0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
### one-hotencoding
encoding_columns = []

#### one-hotencoding処理対象列を取得
for col in df.columns:
    if df[col].dtype=="O":
        print(col)
        encoding_columns.append(col)

NAME_CONTRACT_TYPE
CODE_GENDER
FLAG_OWN_CAR
FLAG_OWN_REALTY
NAME_TYPE_SUITE
NAME_INCOME_TYPE
NAME_EDUCATION_TYPE
NAME_FAMILY_STATUS
NAME_HOUSING_TYPE
OCCUPATION_TYPE
WEEKDAY_APPR_PROCESS_START
ORGANIZATION_TYPE
FONDKAPREMONT_MODE
HOUSETYPE_MODE
WALLSMATERIAL_MODE
EMERGENCYSTATE_MODE


In [8]:
# general cleaning procedures
df = df[df['CODE_GENDER'] != 'XNA']
df = df[df['AMT_INCOME_TOTAL'] < 20000000] # remove a outlier 117M
# NaN values for DAYS_EMPLOYED: 365.243 -> nan
df['DAYS_EMPLOYED'].replace(365243, np.nan, inplace=True) # set null value
df['DAYS_LAST_PHONE_CHANGE'].replace(0, np.nan, inplace=True) # set null value

### CODE_GENDER, 
# Categorical features with Binary encode (0 or 1; two categories)
for bin_feature in ['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY']:
    df[bin_feature], uniques = pd.factorize(df[bin_feature])

### NAME_INCOME_TYPE
df['NAME_INCOME_TYPE'].replace(['Maternity leave'], ['Unemployed'], inplace=True)
df['NAME_INCOME_TYPE'].replace(['Working','Commercial associate','Pensioner','State servant','Unemployed','Student','Businessman','Maternity leave'], [0, 1, 2, 3, 4, 5, 6, 7], inplace=True)
### NAME_FAMILY_STATUS
df['NAME_FAMILY_STATUS'].replace('Unknown', 'Married', inplace=True)
df['NAME_FAMILY_STATUS'].replace(['Married','Single / not married','Civil marriage','Separated','Widow'], [0, 1, 2, 3, 4], inplace=True)

### 
df = df.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '_', x))

df.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,CARD_M3BALANCE_LIMIT_RATIO_MIN,CARD_M3BALANCE_LIMIT_RATIO_MAX,CARD_M3DRAWING_LIMIT_RATIO_MIN,CARD_M3DRAWING_LIMIT_RATIO_MAX,CARD_M3CARD_IS_DPD_MEAN,CARD_M3CARD_IS_DPD_SUM,CARD_M3CARD_IS_DPD_UNDER_120_MEAN,CARD_M3CARD_IS_DPD_UNDER_120_SUM,CARD_M3CARD_IS_DPD_OVER_120_MEAN,CARD_M3CARD_IS_DPD_OVER_120_SUM
0,100002,1.0,Cash loans,0,0,0,0,202500.0,406597.5,24700.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100003,0.0,Cash loans,1,0,1,0,270000.0,1293502.5,35698.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100004,0.0,Revolving loans,0,1,0,0,67500.0,135000.0,6750.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100006,0.0,Cash loans,1,0,0,0,135000.0,312682.5,29686.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,100007,0.0,Cash loans,0,0,0,0,121500.0,513000.0,21865.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
#### one-hotencoding実行
for col in encoding_columns:
    df = pd.get_dummies(df, columns=[col], dummy_na=True)

df.head()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,WALLSMATERIAL_MODE_nan,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes,EMERGENCYSTATE_MODE_nan
0,100002,1.0,0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,-637.0,...,0,0,0,0,1,0,0,1,0,0
1,100003,0.0,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765,-1188.0,...,0,0,0,0,0,0,0,1,0,0
2,100004,0.0,0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046,-225.0,...,0,0,0,0,0,0,1,0,0,1
3,100006,0.0,0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005,-3039.0,...,0,0,0,0,0,0,1,0,0,1
4,100007,0.0,0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932,-3038.0,...,0,0,0,0,0,0,1,0,0,1


In [10]:
### infをnanへ置換
df = df.replace([np.inf, -np.inf], np.nan)
df.head()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,WALLSMATERIAL_MODE_nan,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes,EMERGENCYSTATE_MODE_nan
0,100002,1.0,0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,-637.0,...,0,0,0,0,1,0,0,1,0,0
1,100003,0.0,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765,-1188.0,...,0,0,0,0,0,0,0,1,0,0
2,100004,0.0,0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046,-225.0,...,0,0,0,0,0,0,1,0,0,1
3,100006,0.0,0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005,-3039.0,...,0,0,0,0,0,0,1,0,0,1
4,100007,0.0,0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932,-3038.0,...,0,0,0,0,0,0,1,0,0,1


In [11]:
### 変数選択01
df = FeatureEngineering2.data_post_processing(df)
print('='*50, '\n')
print('---=> df final shape:', df.shape, ' <=---', '\n')
print('=' * 50)
df.head()

---=> the DATA POST-PROCESSING is beginning, the dataset has 603 features
names of feature are renamed
---=> pandas data types of features in the dataset are converted for a reduced memory usage
---=> 593 features are remained after removing non-informative features
60 features are eliminated by LightGBM classifier with an 0.7 auc score limit in step I
---=> 533 features are remained after removing features not interesting for LightGBM classifier
---=> 38 features are generated with the risk_groupanizer
---=> the DATA POST-PROCESSING is ended!, now the dataset has a total 571 features

---=> df final shape: (356250, 571)  <=--- 



,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,POS_M20MONTHS_BALANCE_MAX_high_risk,POS_M20MONTHS_BALANCE_MAX_low_risk,CARD_M3SK_ID_CURR_COUNT_high_risk,CARD_M3SK_ID_CURR_COUNT_low_risk,CARD_M3CARD_IS_DPD_SUM_high_risk,CARD_M3CARD_IS_DPD_SUM_low_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_high_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_low_risk,CARD_M3CARD_IS_DPD_OVER_120_SUM_high_risk,CARD_M3CARD_IS_DPD_OVER_120_SUM_low_risk
0,100002,1.0,0,202500.0,406597.5,24700.5,351000.0,0.018799,-9461,-637.0,...,0,1,0,0,0,0,0,0,0,0
1,100003,0.0,0,270000.0,1293502.5,35698.5,1129500.0,0.003542,-16765,-1188.0,...,1,0,0,0,0,0,0,0,0,0
2,100004,0.0,0,67500.0,135000.0,6750.0,135000.0,0.010033,-19046,-225.0,...,0,0,0,0,0,0,0,0,0,0
3,100006,0.0,0,135000.0,312682.5,29686.5,297000.0,0.008018,-19005,-3040.0,...,0,1,1,0,1,0,1,0,1,0
4,100007,0.0,0,121500.0,513000.0,21865.5,513000.0,0.028656,-19932,-3038.0,...,0,1,0,0,0,0,0,0,0,0


In [12]:
### 変数選択02
df = FeatureEngineering2.data_post_processing(df)
print('='*50, '\n')
print('---=> df final shape:', df.shape, ' <=---', '\n')
print('=' * 50)
df.head()

---=> the DATA POST-PROCESSING is beginning, the dataset has 571 features
names of feature are renamed
---=> pandas data types of features in the dataset are converted for a reduced memory usage
---=> 567 features are remained after removing non-informative features
22 features are eliminated by LightGBM classifier with an 0.7 auc score limit in step I
---=> 545 features are remained after removing features not interesting for LightGBM classifier
---=> 15 features are generated with the risk_groupanizer
---=> the DATA POST-PROCESSING is ended!, now the dataset has a total 560 features

---=> df final shape: (356250, 560)  <=--- 



,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,PREV_PREV_GOODS_DIFF_MAX_low_risk,BUREAU_BUREAU_IS_DPD_SUM_high_risk,BUREAU_BUREAU_IS_DPD_SUM_low_risk,BUREAU_ACT_BUREAU_IS_DPD_SUM_high_risk,CARD_M3SK_ID_CURR_COUNT_high_risk,CARD_M3SK_ID_CURR_COUNT_low_risk,CARD_M3CARD_IS_DPD_SUM_high_risk,CARD_M3CARD_IS_DPD_SUM_low_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_high_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_low_risk
0,100002,1.0,0,202500.0,406597.5,24700.5,351000.0,0.018799,-9461,-637.0,...,0,0,1,1,0,0,0,0,0,0
1,100003,0.0,0,270000.0,1293502.5,35698.5,1129500.0,0.003542,-16765,-1188.0,...,0,0,1,1,0,0,0,0,0,0
2,100004,0.0,0,67500.0,135000.0,6750.0,135000.0,0.010033,-19046,-225.0,...,0,0,1,0,0,0,0,0,0,0
3,100006,0.0,0,135000.0,312682.5,29686.5,297000.0,0.008018,-19005,-3040.0,...,0,0,0,0,1,0,1,0,1,0
4,100007,0.0,0,121500.0,513000.0,21865.5,513000.0,0.028656,-19932,-3038.0,...,0,0,1,0,0,0,0,0,0,0


In [13]:
df_train, df_test = FeatureEngineering3.get_apps_all_train_test(df)
df_train.head()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,PREV_PREV_GOODS_DIFF_MAX_low_risk,BUREAU_BUREAU_IS_DPD_SUM_high_risk,BUREAU_BUREAU_IS_DPD_SUM_low_risk,BUREAU_ACT_BUREAU_IS_DPD_SUM_high_risk,CARD_M3SK_ID_CURR_COUNT_high_risk,CARD_M3SK_ID_CURR_COUNT_low_risk,CARD_M3CARD_IS_DPD_SUM_high_risk,CARD_M3CARD_IS_DPD_SUM_low_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_high_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_low_risk
0,100002,1.0,0,202500.0,406597.5,24700.5,351000.0,0.018799,-9461,-637.0,...,0,0,1,1,0,0,0,0,0,0
1,100003,0.0,0,270000.0,1293502.5,35698.5,1129500.0,0.003542,-16765,-1188.0,...,0,0,1,1,0,0,0,0,0,0
2,100004,0.0,0,67500.0,135000.0,6750.0,135000.0,0.010033,-19046,-225.0,...,0,0,1,0,0,0,0,0,0,0
3,100006,0.0,0,135000.0,312682.5,29686.5,297000.0,0.008018,-19005,-3040.0,...,0,0,0,0,1,0,1,0,1,0
4,100007,0.0,0,121500.0,513000.0,21865.5,513000.0,0.028656,-19932,-3038.0,...,0,0,1,0,0,0,0,0,0,0


In [14]:
df_test.head()

,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,PREV_PREV_GOODS_DIFF_MAX_low_risk,BUREAU_BUREAU_IS_DPD_SUM_high_risk,BUREAU_BUREAU_IS_DPD_SUM_low_risk,BUREAU_ACT_BUREAU_IS_DPD_SUM_high_risk,CARD_M3SK_ID_CURR_COUNT_high_risk,CARD_M3SK_ID_CURR_COUNT_low_risk,CARD_M3CARD_IS_DPD_SUM_high_risk,CARD_M3CARD_IS_DPD_SUM_low_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_high_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_low_risk
307511,100001,0,135000.0,568800.0,20560.5,450000.0,0.018845,-19241,-2328.0,-5168.0,...,0,0,1,1,0,0,0,0,0,0
307512,100005,0,99000.0,222768.0,17370.0,180000.0,0.035797,-18064,-4468.0,-9120.0,...,0,0,1,1,0,0,0,0,0,0
307513,100013,0,202500.0,663264.0,69777.0,630000.0,0.019104,-20038,-4456.0,-2176.0,...,0,0,1,0,1,0,1,0,1,0
307514,100028,2,315000.0,1575000.0,49018.5,1575000.0,0.026398,-13976,-1866.0,-2000.0,...,0,0,1,1,1,0,1,0,1,0
307515,100038,1,180000.0,625500.0,32067.0,625500.0,0.010033,-13040,-2192.0,-4000.0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
train_setB1a = pd.read_csv('./home-credit-default-risk/exports/train_setB1a.csv')
test_setB1a =  pd.read_csv('./home-credit-default-risk/exports/test_setB1a.csv')
test_setB1a.head()

,SK_ID_CURR,setB1a
0,100001,0.033433
1,100005,0.098996
2,100013,0.042185
3,100028,0.039429
4,100038,0.161166


In [16]:
df_train = df_train.merge(train_setB1a, how='left', on='SK_ID_CURR')
df_test = df_test.merge(test_setB1a, how='left', on='SK_ID_CURR')
df_test.head()

,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,BUREAU_BUREAU_IS_DPD_SUM_high_risk,BUREAU_BUREAU_IS_DPD_SUM_low_risk,BUREAU_ACT_BUREAU_IS_DPD_SUM_high_risk,CARD_M3SK_ID_CURR_COUNT_high_risk,CARD_M3SK_ID_CURR_COUNT_low_risk,CARD_M3CARD_IS_DPD_SUM_high_risk,CARD_M3CARD_IS_DPD_SUM_low_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_high_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_low_risk,setB1a
0,100001,0,135000.0,568800.0,20560.5,450000.0,0.018845,-19241,-2328.0,-5168.0,...,0,1,1,0,0,0,0,0,0,0.033433
1,100005,0,99000.0,222768.0,17370.0,180000.0,0.035797,-18064,-4468.0,-9120.0,...,0,1,1,0,0,0,0,0,0,0.098996
2,100013,0,202500.0,663264.0,69777.0,630000.0,0.019104,-20038,-4456.0,-2176.0,...,0,1,0,1,0,1,0,1,0,0.042185
3,100028,2,315000.0,1575000.0,49018.5,1575000.0,0.026398,-13976,-1866.0,-2000.0,...,0,1,1,1,0,1,0,1,0,0.039429
4,100038,1,180000.0,625500.0,32067.0,625500.0,0.010033,-13040,-2192.0,-4000.0,...,0,0,0,0,0,0,0,0,0,0.161166


In [17]:
train_setB1xgb = pd.read_csv('./home-credit-default-risk/exports/train_setB1xgb.csv')
test_setB1xgb =  pd.read_csv('./home-credit-default-risk/exports/test_setB1xgb.csv')
test_setB1xgb.head()

,SK_ID_CURR,setB1x
0,100001,0.032762
1,100005,0.100620
2,100013,0.044999
3,100028,0.034211
4,100038,0.172532


In [18]:
df_train = df_train.merge(train_setB1xgb, how='left', on='SK_ID_CURR')
df_test = df_test.merge(test_setB1xgb, how='left', on='SK_ID_CURR')
df_test.head()

,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,BUREAU_BUREAU_IS_DPD_SUM_low_risk,BUREAU_ACT_BUREAU_IS_DPD_SUM_high_risk,CARD_M3SK_ID_CURR_COUNT_high_risk,CARD_M3SK_ID_CURR_COUNT_low_risk,CARD_M3CARD_IS_DPD_SUM_high_risk,CARD_M3CARD_IS_DPD_SUM_low_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_high_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_low_risk,setB1a,setB1x
0,100001,0,135000.0,568800.0,20560.5,450000.0,0.018845,-19241,-2328.0,-5168.0,...,1,1,0,0,0,0,0,0,0.033433,0.032762
1,100005,0,99000.0,222768.0,17370.0,180000.0,0.035797,-18064,-4468.0,-9120.0,...,1,1,0,0,0,0,0,0,0.098996,0.100620
2,100013,0,202500.0,663264.0,69777.0,630000.0,0.019104,-20038,-4456.0,-2176.0,...,1,0,1,0,1,0,1,0,0.042185,0.044999
3,100028,2,315000.0,1575000.0,49018.5,1575000.0,0.026398,-13976,-1866.0,-2000.0,...,1,1,1,0,1,0,1,0,0.039429,0.034211
4,100038,1,180000.0,625500.0,32067.0,625500.0,0.010033,-13040,-2192.0,-4000.0,...,0,0,0,0,0,0,0,0,0.161166,0.172532


In [19]:
pred_train_set1a = pd.read_csv('./home-credit-default-risk/exports/pred_train_set1a.csv')
pred_test_set1a =  pd.read_csv('./home-credit-default-risk/exports/pred_test_set1a.csv')
pred_test_set1a.head()

,SK_ID_CURR,set1
0,100001,0.040068
1,100005,0.095240
2,100013,0.044435
3,100028,0.036586
4,100038,0.141754


In [20]:
df_train = df_train.merge(pred_train_set1a, how='left', on='SK_ID_CURR')
df_test = df_test.merge(pred_test_set1a, how='left', on='SK_ID_CURR')
df_test.head()

,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,BUREAU_ACT_BUREAU_IS_DPD_SUM_high_risk,CARD_M3SK_ID_CURR_COUNT_high_risk,CARD_M3SK_ID_CURR_COUNT_low_risk,CARD_M3CARD_IS_DPD_SUM_high_risk,CARD_M3CARD_IS_DPD_SUM_low_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_high_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_low_risk,setB1a,setB1x,set1
0,100001,0,135000.0,568800.0,20560.5,450000.0,0.018845,-19241,-2328.0,-5168.0,...,1,0,0,0,0,0,0,0.033433,0.032762,0.040068
1,100005,0,99000.0,222768.0,17370.0,180000.0,0.035797,-18064,-4468.0,-9120.0,...,1,0,0,0,0,0,0,0.098996,0.100620,0.095240
2,100013,0,202500.0,663264.0,69777.0,630000.0,0.019104,-20038,-4456.0,-2176.0,...,0,1,0,1,0,1,0,0.042185,0.044999,0.044435
3,100028,2,315000.0,1575000.0,49018.5,1575000.0,0.026398,-13976,-1866.0,-2000.0,...,1,1,0,1,0,1,0,0.039429,0.034211,0.036586
4,100038,1,180000.0,625500.0,32067.0,625500.0,0.010033,-13040,-2192.0,-4000.0,...,0,0,0,0,0,0,0,0.161166,0.172532,0.141754


In [21]:
train_set1xgb = pd.read_csv('./home-credit-default-risk/exports/train_set1xgb.csv')
test_set1xgb =  pd.read_csv('./home-credit-default-risk/exports/test_set1xgb.csv')
test_set1xgb.head()

,SK_ID_CURR,set1x
0,100001,0.035234
1,100005,0.107426
2,100013,0.048029
3,100028,0.032975
4,100038,0.161962


In [22]:
df_train = df_train.merge(train_set1xgb, how='left', on='SK_ID_CURR')
df_test = df_test.merge(test_set1xgb, how='left', on='SK_ID_CURR')
df_test.head()

,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,CARD_M3SK_ID_CURR_COUNT_high_risk,CARD_M3SK_ID_CURR_COUNT_low_risk,CARD_M3CARD_IS_DPD_SUM_high_risk,CARD_M3CARD_IS_DPD_SUM_low_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_high_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_low_risk,setB1a,setB1x,set1,set1x
0,100001,0,135000.0,568800.0,20560.5,450000.0,0.018845,-19241,-2328.0,-5168.0,...,0,0,0,0,0,0,0.033433,0.032762,0.040068,0.035234
1,100005,0,99000.0,222768.0,17370.0,180000.0,0.035797,-18064,-4468.0,-9120.0,...,0,0,0,0,0,0,0.098996,0.100620,0.095240,0.107426
2,100013,0,202500.0,663264.0,69777.0,630000.0,0.019104,-20038,-4456.0,-2176.0,...,1,0,1,0,1,0,0.042185,0.044999,0.044435,0.048029
3,100028,2,315000.0,1575000.0,49018.5,1575000.0,0.026398,-13976,-1866.0,-2000.0,...,1,0,1,0,1,0,0.039429,0.034211,0.036586,0.032975
4,100038,1,180000.0,625500.0,32067.0,625500.0,0.010033,-13040,-2192.0,-4000.0,...,0,0,0,0,0,0,0.161166,0.172532,0.141754,0.161962


In [23]:
train_set2a = pd.read_csv('./home-credit-default-risk/exports/train_set2a.csv')
test_set2a =  pd.read_csv('./home-credit-default-risk/exports/test_set2a.csv')
test_set2a.head()

,SK_ID_CURR,set2a
0,100001,0.042769
1,100005,0.103694
2,100013,0.036006
3,100028,0.048812
4,100038,0.173045


In [24]:
df_train = df_train.merge(train_set2a, how='left', on='SK_ID_CURR')
df_test = df_test.merge(test_set2a, how='left', on='SK_ID_CURR')
df_test.head()

,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,CARD_M3SK_ID_CURR_COUNT_low_risk,CARD_M3CARD_IS_DPD_SUM_high_risk,CARD_M3CARD_IS_DPD_SUM_low_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_high_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_low_risk,setB1a,setB1x,set1,set1x,set2a
0,100001,0,135000.0,568800.0,20560.5,450000.0,0.018845,-19241,-2328.0,-5168.0,...,0,0,0,0,0,0.033433,0.032762,0.040068,0.035234,0.042769
1,100005,0,99000.0,222768.0,17370.0,180000.0,0.035797,-18064,-4468.0,-9120.0,...,0,0,0,0,0,0.098996,0.100620,0.095240,0.107426,0.103694
2,100013,0,202500.0,663264.0,69777.0,630000.0,0.019104,-20038,-4456.0,-2176.0,...,0,1,0,1,0,0.042185,0.044999,0.044435,0.048029,0.036006
3,100028,2,315000.0,1575000.0,49018.5,1575000.0,0.026398,-13976,-1866.0,-2000.0,...,0,1,0,1,0,0.039429,0.034211,0.036586,0.032975,0.048812
4,100038,1,180000.0,625500.0,32067.0,625500.0,0.010033,-13040,-2192.0,-4000.0,...,0,0,0,0,0,0.161166,0.172532,0.141754,0.161962,0.173045


In [25]:
train_set2xgb = pd.read_csv('./home-credit-default-risk/exports/train_set2xgb.csv')
test_set2xgb =  pd.read_csv('./home-credit-default-risk/exports/test_set2xgb.csv')
test_set2xgb.head()

,SK_ID_CURR,set2x
0,100001,0.035013
1,100005,0.103532
2,100013,0.048415
3,100028,0.032249
4,100038,0.150442


In [26]:
df_train = df_train.merge(train_set2xgb, how='left', on='SK_ID_CURR')
df_test = df_test.merge(test_set2xgb, how='left', on='SK_ID_CURR')
df_test.head()

,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,CARD_M3CARD_IS_DPD_SUM_high_risk,CARD_M3CARD_IS_DPD_SUM_low_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_high_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_low_risk,setB1a,setB1x,set1,set1x,set2a,set2x
0,100001,0,135000.0,568800.0,20560.5,450000.0,0.018845,-19241,-2328.0,-5168.0,...,0,0,0,0,0.033433,0.032762,0.040068,0.035234,0.042769,0.035013
1,100005,0,99000.0,222768.0,17370.0,180000.0,0.035797,-18064,-4468.0,-9120.0,...,0,0,0,0,0.098996,0.100620,0.095240,0.107426,0.103694,0.103532
2,100013,0,202500.0,663264.0,69777.0,630000.0,0.019104,-20038,-4456.0,-2176.0,...,1,0,1,0,0.042185,0.044999,0.044435,0.048029,0.036006,0.048415
3,100028,2,315000.0,1575000.0,49018.5,1575000.0,0.026398,-13976,-1866.0,-2000.0,...,1,0,1,0,0.039429,0.034211,0.036586,0.032975,0.048812,0.032249
4,100038,1,180000.0,625500.0,32067.0,625500.0,0.010033,-13040,-2192.0,-4000.0,...,0,0,0,0,0.161166,0.172532,0.141754,0.161962,0.173045,0.150442


In [27]:
train_set2nn = pd.read_csv('./home-credit-default-risk/exports/train_set2nn.csv')
test_set2nn =  pd.read_csv('./home-credit-default-risk/exports/test_set2nn.csv')
test_set2nn.head()

,SK_ID_CURR,set2nn
0,100001,0.048972
1,100005,0.141351
2,100013,0.028088
3,100028,0.039056
4,100038,0.192637


In [28]:
df_train = df_train.merge(train_set2nn, how='left', on='SK_ID_CURR')
df_test = df_test.merge(test_set2nn, how='left', on='SK_ID_CURR')
df_test.head()

,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,CARD_M3CARD_IS_DPD_SUM_low_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_high_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_low_risk,setB1a,setB1x,set1,set1x,set2a,set2x,set2nn
0,100001,0,135000.0,568800.0,20560.5,450000.0,0.018845,-19241,-2328.0,-5168.0,...,0,0,0,0.033433,0.032762,0.040068,0.035234,0.042769,0.035013,0.048972
1,100005,0,99000.0,222768.0,17370.0,180000.0,0.035797,-18064,-4468.0,-9120.0,...,0,0,0,0.098996,0.100620,0.095240,0.107426,0.103694,0.103532,0.141351
2,100013,0,202500.0,663264.0,69777.0,630000.0,0.019104,-20038,-4456.0,-2176.0,...,0,1,0,0.042185,0.044999,0.044435,0.048029,0.036006,0.048415,0.028088
3,100028,2,315000.0,1575000.0,49018.5,1575000.0,0.026398,-13976,-1866.0,-2000.0,...,0,1,0,0.039429,0.034211,0.036586,0.032975,0.048812,0.032249,0.039056
4,100038,1,180000.0,625500.0,32067.0,625500.0,0.010033,-13040,-2192.0,-4000.0,...,0,0,0,0.161166,0.172532,0.141754,0.161962,0.173045,0.150442,0.192637


In [29]:
train_set3xgb = pd.read_csv('./home-credit-default-risk/exports/train_set3xgb.csv')
test_set3xgb =  pd.read_csv('./home-credit-default-risk/exports/test_set3xgb.csv')
test_set3xgb.head()

,SK_ID_CURR,set3x
0,100001,0.034310
1,100005,0.127556
2,100013,0.044872
3,100028,0.030915
4,100038,0.201295


In [30]:
#df_train = df_train.merge(train_set3xgb, how='left', on='SK_ID_CURR')
#df_test = df_test.merge(test_set3xgb, how='left', on='SK_ID_CURR')
df_test.head()

,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,CARD_M3CARD_IS_DPD_SUM_low_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_high_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_low_risk,setB1a,setB1x,set1,set1x,set2a,set2x,set2nn
0,100001,0,135000.0,568800.0,20560.5,450000.0,0.018845,-19241,-2328.0,-5168.0,...,0,0,0,0.033433,0.032762,0.040068,0.035234,0.042769,0.035013,0.048972
1,100005,0,99000.0,222768.0,17370.0,180000.0,0.035797,-18064,-4468.0,-9120.0,...,0,0,0,0.098996,0.100620,0.095240,0.107426,0.103694,0.103532,0.141351
2,100013,0,202500.0,663264.0,69777.0,630000.0,0.019104,-20038,-4456.0,-2176.0,...,0,1,0,0.042185,0.044999,0.044435,0.048029,0.036006,0.048415,0.028088
3,100028,2,315000.0,1575000.0,49018.5,1575000.0,0.026398,-13976,-1866.0,-2000.0,...,0,1,0,0.039429,0.034211,0.036586,0.032975,0.048812,0.032249,0.039056
4,100038,1,180000.0,625500.0,32067.0,625500.0,0.010033,-13040,-2192.0,-4000.0,...,0,0,0,0.161166,0.172532,0.141754,0.161962,0.173045,0.150442,0.192637


In [31]:
train_set4a = pd.read_csv('./home-credit-default-risk/exports/train_set4a.csv')
test_set4a =  pd.read_csv('./home-credit-default-risk/exports/test_set4a.csv')
test_set4a.head()

,SK_ID_CURR,set4a
0,100001,0.042113
1,100005,0.132192
2,100013,0.057271
3,100028,0.042206
4,100038,0.195048


In [32]:
df_train = df_train.merge(train_set4a, how='left', on='SK_ID_CURR')
df_test = df_test.merge(test_set4a, how='left', on='SK_ID_CURR')
df_test.head()

,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_high_risk,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_low_risk,setB1a,setB1x,set1,set1x,set2a,set2x,set2nn,set4a
0,100001,0,135000.0,568800.0,20560.5,450000.0,0.018845,-19241,-2328.0,-5168.0,...,0,0,0.033433,0.032762,0.040068,0.035234,0.042769,0.035013,0.048972,0.042113
1,100005,0,99000.0,222768.0,17370.0,180000.0,0.035797,-18064,-4468.0,-9120.0,...,0,0,0.098996,0.100620,0.095240,0.107426,0.103694,0.103532,0.141351,0.132192
2,100013,0,202500.0,663264.0,69777.0,630000.0,0.019104,-20038,-4456.0,-2176.0,...,1,0,0.042185,0.044999,0.044435,0.048029,0.036006,0.048415,0.028088,0.057271
3,100028,2,315000.0,1575000.0,49018.5,1575000.0,0.026398,-13976,-1866.0,-2000.0,...,1,0,0.039429,0.034211,0.036586,0.032975,0.048812,0.032249,0.039056,0.042206
4,100038,1,180000.0,625500.0,32067.0,625500.0,0.010033,-13040,-2192.0,-4000.0,...,0,0,0.161166,0.172532,0.141754,0.161962,0.173045,0.150442,0.192637,0.195048


In [33]:
train_set4xgb = pd.read_csv('./home-credit-default-risk/exports/train_set4xgb.csv')
test_set4xgb =  pd.read_csv('./home-credit-default-risk/exports/test_set4xgb.csv')
test_set4xgb.head()

,SK_ID_CURR,set4x
0,100001,0.034978
1,100005,0.118268
2,100013,0.039774
3,100028,0.037283
4,100038,0.164890


In [34]:
df_train = df_train.merge(train_set4xgb, how='left', on='SK_ID_CURR')
df_test = df_test.merge(test_set4xgb, how='left', on='SK_ID_CURR')
df_test.head()

,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,CARD_M3CARD_IS_DPD_UNDER_120_MEAN_low_risk,setB1a,setB1x,set1,set1x,set2a,set2x,set2nn,set4a,set4x
0,100001,0,135000.0,568800.0,20560.5,450000.0,0.018845,-19241,-2328.0,-5168.0,...,0,0.033433,0.032762,0.040068,0.035234,0.042769,0.035013,0.048972,0.042113,0.034978
1,100005,0,99000.0,222768.0,17370.0,180000.0,0.035797,-18064,-4468.0,-9120.0,...,0,0.098996,0.100620,0.095240,0.107426,0.103694,0.103532,0.141351,0.132192,0.118268
2,100013,0,202500.0,663264.0,69777.0,630000.0,0.019104,-20038,-4456.0,-2176.0,...,0,0.042185,0.044999,0.044435,0.048029,0.036006,0.048415,0.028088,0.057271,0.039774
3,100028,2,315000.0,1575000.0,49018.5,1575000.0,0.026398,-13976,-1866.0,-2000.0,...,0,0.039429,0.034211,0.036586,0.032975,0.048812,0.032249,0.039056,0.042206,0.037283
4,100038,1,180000.0,625500.0,32067.0,625500.0,0.010033,-13040,-2192.0,-4000.0,...,0,0.161166,0.172532,0.141754,0.161962,0.173045,0.150442,0.192637,0.195048,0.164890


In [35]:
train_set5a = pd.read_csv('./home-credit-default-risk/exports/train_set5a.csv')
test_set5a =  pd.read_csv('./home-credit-default-risk/exports/test_set5a.csv')
test_set4a.head()

,SK_ID_CURR,set4a
0,100001,0.042113
1,100005,0.132192
2,100013,0.057271
3,100028,0.042206
4,100038,0.195048


In [36]:
df_train = df_train.merge(train_set5a, how='left', on='SK_ID_CURR')
df_test = df_test.merge(test_set5a, how='left', on='SK_ID_CURR')
df_test.head()

,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,setB1a,setB1x,set1,set1x,set2a,set2x,set2nn,set4a,set4x,set5a
0,100001,0,135000.0,568800.0,20560.5,450000.0,0.018845,-19241,-2328.0,-5168.0,...,0.033433,0.032762,0.040068,0.035234,0.042769,0.035013,0.048972,0.042113,0.034978,0.034370
1,100005,0,99000.0,222768.0,17370.0,180000.0,0.035797,-18064,-4468.0,-9120.0,...,0.098996,0.100620,0.095240,0.107426,0.103694,0.103532,0.141351,0.132192,0.118268,0.116919
2,100013,0,202500.0,663264.0,69777.0,630000.0,0.019104,-20038,-4456.0,-2176.0,...,0.042185,0.044999,0.044435,0.048029,0.036006,0.048415,0.028088,0.057271,0.039774,0.037191
3,100028,2,315000.0,1575000.0,49018.5,1575000.0,0.026398,-13976,-1866.0,-2000.0,...,0.039429,0.034211,0.036586,0.032975,0.048812,0.032249,0.039056,0.042206,0.037283,0.051958
4,100038,1,180000.0,625500.0,32067.0,625500.0,0.010033,-13040,-2192.0,-4000.0,...,0.161166,0.172532,0.141754,0.161962,0.173045,0.150442,0.192637,0.195048,0.164890,0.141608


In [37]:
train_set5xgb = pd.read_csv('./home-credit-default-risk/exports/train_set5xgb.csv')
test_set5xgb =  pd.read_csv('./home-credit-default-risk/exports/test_set5xgb.csv')
test_set5xgb.head()

,SK_ID_CURR,set5x
0,100001,0.028969
1,100005,0.116288
2,100013,0.041057
3,100028,0.045375
4,100038,0.136362


In [38]:
df_train = df_train.merge(train_set5xgb, how='left', on='SK_ID_CURR')
df_test = df_test.merge(test_set5xgb, how='left', on='SK_ID_CURR')
df_test.head()

,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,setB1x,set1,set1x,set2a,set2x,set2nn,set4a,set4x,set5a,set5x
0,100001,0,135000.0,568800.0,20560.5,450000.0,0.018845,-19241,-2328.0,-5168.0,...,0.032762,0.040068,0.035234,0.042769,0.035013,0.048972,0.042113,0.034978,0.034370,0.028969
1,100005,0,99000.0,222768.0,17370.0,180000.0,0.035797,-18064,-4468.0,-9120.0,...,0.100620,0.095240,0.107426,0.103694,0.103532,0.141351,0.132192,0.118268,0.116919,0.116288
2,100013,0,202500.0,663264.0,69777.0,630000.0,0.019104,-20038,-4456.0,-2176.0,...,0.044999,0.044435,0.048029,0.036006,0.048415,0.028088,0.057271,0.039774,0.037191,0.041057
3,100028,2,315000.0,1575000.0,49018.5,1575000.0,0.026398,-13976,-1866.0,-2000.0,...,0.034211,0.036586,0.032975,0.048812,0.032249,0.039056,0.042206,0.037283,0.051958,0.045375
4,100038,1,180000.0,625500.0,32067.0,625500.0,0.010033,-13040,-2192.0,-4000.0,...,0.172532,0.141754,0.161962,0.173045,0.150442,0.192637,0.195048,0.164890,0.141608,0.136362


In [39]:
### infをnanへ置換
df_train = df_train.replace([np.inf, -np.inf], np.nan)
df_train.head()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,setB1x,set1,set1x,set2a,set2x,set2nn,set4a,set4x,set5a,set5x
0,100002,1.0,0,202500.0,406597.5,24700.5,351000.0,0.018799,-9461,-637.0,...,0.234856,0.260928,0.199398,0.242499,0.198794,0.240473,0.218256,0.207998,0.196429,0.244814
1,100003,0.0,0,270000.0,1293502.5,35698.5,1129500.0,0.003542,-16765,-1188.0,...,0.023065,0.012171,0.012061,0.017857,0.012648,0.014736,0.012283,0.013902,0.013261,0.014884
2,100004,0.0,0,67500.0,135000.0,6750.0,135000.0,0.010033,-19046,-225.0,...,0.038719,0.039950,0.047440,0.033203,0.033060,0.029793,0.033160,0.034973,0.030715,0.036912
3,100006,0.0,0,135000.0,312682.5,29686.5,297000.0,0.008018,-19005,-3040.0,...,0.034008,0.030776,0.030071,0.048900,0.025442,0.031804,0.029616,0.040586,0.034176,0.037270
4,100007,0.0,0,121500.0,513000.0,21865.5,513000.0,0.028656,-19932,-3038.0,...,0.027813,0.035088,0.023555,0.043556,0.039186,0.060946,0.035875,0.036123,0.038256,0.035143


In [40]:
### infをnanへ置換
df_test = df_test.replace([np.inf, -np.inf], np.nan)
df_test.head()

,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,setB1x,set1,set1x,set2a,set2x,set2nn,set4a,set4x,set5a,set5x
0,100001,0,135000.0,568800.0,20560.5,450000.0,0.018845,-19241,-2328.0,-5168.0,...,0.032762,0.040068,0.035234,0.042769,0.035013,0.048972,0.042113,0.034978,0.034370,0.028969
1,100005,0,99000.0,222768.0,17370.0,180000.0,0.035797,-18064,-4468.0,-9120.0,...,0.100620,0.095240,0.107426,0.103694,0.103532,0.141351,0.132192,0.118268,0.116919,0.116288
2,100013,0,202500.0,663264.0,69777.0,630000.0,0.019104,-20038,-4456.0,-2176.0,...,0.044999,0.044435,0.048029,0.036006,0.048415,0.028088,0.057271,0.039774,0.037191,0.041057
3,100028,2,315000.0,1575000.0,49018.5,1575000.0,0.026398,-13976,-1866.0,-2000.0,...,0.034211,0.036586,0.032975,0.048812,0.032249,0.039056,0.042206,0.037283,0.051958,0.045375
4,100038,1,180000.0,625500.0,32067.0,625500.0,0.010033,-13040,-2192.0,-4000.0,...,0.172532,0.141754,0.161962,0.173045,0.150442,0.192637,0.195048,0.164890,0.141608,0.136362


In [41]:
### 処理対象の列名を配列化
#encoding_columns = []
encoding_columns = list(df_test.select_dtypes(include='number').columns)
encoding_columns.remove('SK_ID_CURR')

In [42]:
### indexを設定
df_train = df_train.set_index('SK_ID_CURR')
df_test = df_test.set_index('SK_ID_CURR')
df_train.head()

,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,setB1x,set1,set1x,set2a,set2x,set2nn,set4a,set4x,set5a,set5x
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100002,1.0,0,202500.0,406597.5,24700.5,351000.0,0.018799,-9461,-637.0,-3648.0,...,0.234856,0.260928,0.199398,0.242499,0.198794,0.240473,0.218256,0.207998,0.196429,0.244814
100003,0.0,0,270000.0,1293502.5,35698.5,1129500.0,0.003542,-16765,-1188.0,-1186.0,...,0.023065,0.012171,0.012061,0.017857,0.012648,0.014736,0.012283,0.013902,0.013261,0.014884
100004,0.0,0,67500.0,135000.0,6750.0,135000.0,0.010033,-19046,-225.0,-4260.0,...,0.038719,0.039950,0.047440,0.033203,0.033060,0.029793,0.033160,0.034973,0.030715,0.036912
100006,0.0,0,135000.0,312682.5,29686.5,297000.0,0.008018,-19005,-3040.0,-9832.0,...,0.034008,0.030776,0.030071,0.048900,0.025442,0.031804,0.029616,0.040586,0.034176,0.037270
100007,0.0,0,121500.0,513000.0,21865.5,513000.0,0.028656,-19932,-3038.0,-4312.0,...,0.027813,0.035088,0.023555,0.043556,0.039186,0.060946,0.035875,0.036123,0.038256,0.035143


In [43]:
### 標準化
for column in encoding_columns:
    num_cols = []
    num_cols.append(column)
    result = modules.process_standardization(df_train, df_test, num_cols)
        
df_train.head()

,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,setB1x,set1,set1x,set2a,set2x,set2nn,set4a,set4x,set5a,set5x
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100002,1.0,-0.577531,0.313221,-0.478099,-0.166152,-0.507239,-0.149780,1.506891,0.747070,0.379883,...,1.635142,1.896448,1.301762,1.758129,1.262144,1.746196,1.449139,1.374899,1.193338,1.755504
100003,0.0,-0.577531,0.933556,1.725433,0.592656,1.599963,-1.252930,-0.166811,0.511719,1.079102,...,-0.587670,-0.711865,-0.698289,-0.672096,-0.692681,-0.686084,-0.715210,-0.685605,-0.693219,-0.662952
100004,0.0,-0.577531,-0.927449,-1.152888,-1.404647,-1.091896,-0.783203,-0.689500,0.923828,0.206055,...,-0.423376,-0.420592,-0.320577,-0.506078,-0.478319,-0.523846,-0.495838,-0.461924,-0.513453,-0.431259
100006,0.0,-0.577531,-0.307114,-0.711432,0.177857,-0.653403,-0.928711,-0.680105,-0.280518,-1.375000,...,-0.472816,-0.516790,-0.506012,-0.336265,-0.558316,-0.502178,-0.533075,-0.402327,-0.477805,-0.427493
100007,0.0,-0.577531,-0.431181,-0.213740,-0.361753,-0.068746,0.562988,-0.892525,-0.279785,0.191284,...,-0.537838,-0.471570,-0.575581,-0.394081,-0.413984,-0.188179,-0.467308,-0.449710,-0.435785,-0.449868


In [44]:
### indexを削除
df_train = df_train.reset_index()
df_test = df_test.reset_index()
df_train.head()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,setB1x,set1,set1x,set2a,set2x,set2nn,set4a,set4x,set5a,set5x
0,100002,1.0,-0.577531,0.313221,-0.478099,-0.166152,-0.507239,-0.149780,1.506891,0.747070,...,1.635142,1.896448,1.301762,1.758129,1.262144,1.746196,1.449139,1.374899,1.193338,1.755504
1,100003,0.0,-0.577531,0.933556,1.725433,0.592656,1.599963,-1.252930,-0.166811,0.511719,...,-0.587670,-0.711865,-0.698289,-0.672096,-0.692681,-0.686084,-0.715210,-0.685605,-0.693219,-0.662952
2,100004,0.0,-0.577531,-0.927449,-1.152888,-1.404647,-1.091896,-0.783203,-0.689500,0.923828,...,-0.423376,-0.420592,-0.320577,-0.506078,-0.478319,-0.523846,-0.495838,-0.461924,-0.513453,-0.431259
3,100006,0.0,-0.577531,-0.307114,-0.711432,0.177857,-0.653403,-0.928711,-0.680105,-0.280518,...,-0.472816,-0.516790,-0.506012,-0.336265,-0.558316,-0.502178,-0.533075,-0.402327,-0.477805,-0.427493
4,100007,0.0,-0.577531,-0.431181,-0.213740,-0.361753,-0.068746,0.562988,-0.892525,-0.279785,...,-0.537838,-0.471570,-0.575581,-0.394081,-0.413984,-0.188179,-0.467308,-0.449710,-0.435785,-0.449868


In [45]:
### メモリサイズを縮小
df_train = baseline.reduce_mem_usage(df_train)
df_test = baseline.reduce_mem_usage(df_test)
df_train.head()

Memory usage of dataframe is 714.38 MB
Memory usage of optimization is 702343832.00 MB
Decreased by -98314662.4%
Memory usage of dataframe is 113.15 MB
Memory usage of optimization is 111136448.00 MB
Decreased by -98223131.5%


,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,setB1x,set1,set1x,set2a,set2x,set2nn,set4a,set4x,set5a,set5x
0,100002,1.0,-0.577637,0.313232,-0.478027,-0.166138,-0.507324,-0.149780,1.506836,0.747070,...,1.634766,1.896484,1.301758,1.757812,1.261719,1.746094,1.449219,1.375000,1.193359,1.755859
1,100003,0.0,-0.577637,0.933594,1.725586,0.592773,1.599609,-1.252930,-0.166870,0.511719,...,-0.587891,-0.711914,-0.698242,-0.671875,-0.692871,-0.686035,-0.715332,-0.685547,-0.693359,-0.663086
2,100004,0.0,-0.577637,-0.927246,-1.153320,-1.404297,-1.091797,-0.783203,-0.689453,0.923828,...,-0.423340,-0.420654,-0.320557,-0.505859,-0.478271,-0.523926,-0.495850,-0.461914,-0.513672,-0.431152
3,100006,0.0,-0.577637,-0.307129,-0.711426,0.177856,-0.653320,-0.928711,-0.680176,-0.280518,...,-0.472900,-0.516602,-0.505859,-0.336182,-0.558105,-0.501953,-0.533203,-0.402344,-0.477783,-0.427490
4,100007,0.0,-0.577637,-0.431152,-0.213745,-0.361816,-0.068726,0.562988,-0.892578,-0.279785,...,-0.537598,-0.471680,-0.575684,-0.394043,-0.414062,-0.188232,-0.467285,-0.449707,-0.435791,-0.449951


In [46]:
x_train = df_train.drop(columns=['TARGET', 'SK_ID_CURR'])
y_train = df_train['TARGET']
id_train = df_train[['SK_ID_CURR']]

for col in x_train.columns:
    if x_train[col].dtype=='O':
        x_train[col] = x_train[col].astype('category')

In [47]:
# データセット作成
x_test = df_test.drop(columns=['SK_ID_CURR'])
id_test = df_test[['SK_ID_CURR']]

# カテゴリ変数をcategory型へ変換
for col in x_test.columns:
    if x_test[col].dtype=="O":
        x_test[col] = x_test[col].astype('category')

In [48]:
train_oof, imp, metrics = baseline.train_lgb(x_train
                                    ,y_train
                                    ,id_train
                                    ,params
                                    ,list_nfold=[0,1,2,3,4]
                                    ,n_splits=5)

-------------------- 0 --------------------
(246004, 569) (61502, 569)
[LightGBM] [Warning] num_threads is set with n_jobs=-1, nthread=4 will be ignored. Current value: num_threads=-1
[100]	training's auc: 0.803417	valid_1's auc: 0.794524
[200]	training's auc: 0.805633	valid_1's auc: 0.79448
[300]	training's auc: 0.808055	valid_1's auc: 0.794355
[400]	training's auc: 0.810772	valid_1's auc: 0.794338
[500]	training's auc: 0.813429	valid_1's auc: 0.794347
[600]	training's auc: 0.816236	valid_1's auc: 0.794439
[700]	training's auc: 0.81925	valid_1's auc: 0.794462
[800]	training's auc: 0.822203	valid_1's auc: 0.794506
[900]	training's auc: 0.824656	valid_1's auc: 0.794505
[1000]	training's auc: 0.826951	valid_1's auc: 0.794564
[1100]	training's auc: 0.82891	valid_1's auc: 0.794542
[1200]	training's auc: 0.830955	valid_1's auc: 0.79454
[1300]	training's auc: 0.83328	valid_1's auc: 0.794544
[1400]	training's auc: 0.835427	valid_1's auc: 0.794564
[1500]	training's auc: 0.837566	valid_1's auc:

In [49]:
imp.sort_values("imp", ascending=False)[:20]

,col,imp,imp_std
560,set2a,2710.2,51.050955
562,set2x,1941.4,77.873616
558,set1,1793.0,107.161094
564,set4x,943.6,43.661196
566,set5x,909.8,71.180756
21,APPS_ANNUITY_CREDIT_RATIO,882.4,60.616004
559,set1x,861.6,51.713635
232,DAYS_BIRTH,822.0,21.874643
48,APP_SCORE2_TO_PHONE_CHANGE_RATIO,781.6,48.849770
235,DAYS_LAST_PHONE_CHANGE,746.4,44.958870


In [50]:
# データセット作成
x_test = df_test.drop(columns=['SK_ID_CURR'])
id_test = df_test[['SK_ID_CURR']]

# カテゴリ変数をcategory型へ変換
for col in x_test.columns:
    if x_test[col].dtype=="O":
        x_test[col] = x_test[col].astype('category')

In [51]:
test_pred = baseline.predict_lgb(x_test
                                ,id_test
                                ,list_nfold=[0,1,2,3,4])

-------------------- 0 --------------------
-------------------- 1 --------------------
-------------------- 2 --------------------
-------------------- 3 --------------------
-------------------- 4 --------------------
Done.


In [52]:
df_submit = test_pred.rename(columns={"pred":"TARGET"})
print(df_submit.shape)
display(df_submit.head())
df_submit.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submission_OS_3-1.csv" # 出力先
    ,index=None                                        # indexの出力有無
)

(48744, 2)


,SK_ID_CURR,TARGET
0,100001,0.036921
1,100005,0.115780
2,100013,0.058753
3,100028,0.042459
4,100038,0.157480


## パラメータ・チューニング

In [53]:
#baseline = Baseline('SK_ID_CURR')

#sampler = optuna.samplers.TPESampler(seed=123)
#study = optuna.create_study(sampler=sampler, direction='maximize')
#study.optimize(objective, n_trials=50, n_jobs=5)

In [54]:
#trial = study.best_trial
#print("auc(best)={:.4f}".format(trial.value))
#display(trial.params)

In [55]:
#params_best = trial.params
#params_best.update(params_base)
#display(params_best)

In [56]:
#train_oof, imp, metrics = baseline.train_lgb(x_train
#                                    ,y_train
#                                    ,id_train
#                                    ,list_nfold=[0,1,2,3,4]
#                                    ,n_splits=5
#                                    ,params=params_best)

In [57]:
### 推論用データセットの作成
#x_test = df_test.drop(columns=['SK_ID_CURR'])
#id_test = df_test[['SK_ID_CURR']]

### カテゴリ変数をcategory型へ変換
#for col in x_test.columns:
#    if x_test[col].dtype=="O":
#        x_test[col] = x_test[col].astype('category')

### predict
#test_pred = baseline.predict_lgb(x_test
#                       ,id_test
#                       ,list_nfold=[0,1,2,3,4])

### make submission-file
#df_submit = test_pred.rename(columns={"pred":"TARGET"})
#print(df_submit.shape)
#display(df_submit.head())
#df_submit.to_csv("./home-credit-default-risk/submit/submission_OS_3-2.csv", index=None)